To evaluate simplification quality 2 baselines will be used:

* Simple trunctuation baseline: leave only a certain number of the first words of a sentence
* GPT generation (finetuned russian gpt-3 model from Sber)

As the test set I will use the dev part of the Russian dataset collected via Toloka

# Trunctuation Baseline

##Loading data...

In [ ]:
# dependencies
! pip install textstat
! pip install --upgrade language_tool_python

In [2]:
import nltk
import textstat
import language_tool_python
nltk.download('punkt')
import re
import pandas as pd
import codecs
from collections import OrderedDict
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# test_data2 = pd.read_csv('/content/drive/MyDrive/SImplification_models/hidden_test_sents.csv')
# test_data = pd.read_csv('/content/drive/MyDrive/SImplification_models/public_test_sents.csv')#, error_bad_lines=False, engine='python')
# full_test = pd.concat((test_data, test_data2))
# test_data.head()

In [ ]:
! wget https://raw.githubusercontent.com/dialogue-evaluation/RuSimpleSentEval/main/dev_sents.csv
test_data = pd.read_csv('/content/dev_sents.csv')
test_data.shape[0]

First look at the test data (which is actually a dev part from the contest)

In [ ]:
test_data['trunctuation_bs'] = test_data['INPUT:source'].apply(lambda x: ' '.join(x.split()[:int(len(x.split())*0.6)])+'.')
test_data.sample(5)

,Unnamed: 0,INPUT:source,OUTPUT:output,trunctuation_bs
225,686,"В Китае передача идёт в основном в кругу семьи, внутрибольничная передача в данной стране для инфекции не характерна.","В Китае обычно заражение происходит от родственников, а не из-за посещения больницы.","В Китае передача идёт в основном в кругу семьи, внутрибольничная."
1073,3229,"За столетия потрясений численность этнических вавилонян в Южной Месопотамии сократилась, большинство населения там составляли халдеи.",За последние несколько столетий сократилось население вавилонян в Южной Месопотамии,За столетия потрясений численность этнических вавилонян в Южной Месопотамии.
1391,4178,"Красный Крест сообщил, что не может начать работу в Карабахе из-за нарушений перемирия.",Работа красного креста не возможра в аарабахе по причине нарушения перемирия.,"Красный Крест сообщил, что не может начать."
3146,9234,"Туристическая индустрия Ирана серьёзно пострадала в результате ирано-иракской войны, однако в настоящее время возрождается.","Туриндустрия Ирана пострадала в результате ирано-иракской войны, но сейчас возрождается.",Туристическая индустрия Ирана серьёзно пострадала в результате ирано-иракской.
844,2478,"Гениальнейшим поэтом Польши и одновременно одним из великих мировых поэтов является Адам Мицкевич, признанный вождь польского романтизма.",Адам Мицкевич - всемирно известный польский поэт.\r\n,Гениальнейшим поэтом Польши и одновременно одним из великих мировых поэтов.


# Second baseline will be automatically generated by GPT-3

I follow the procedure described in Sber repo
# Finetune RuGPTs in megatron without deepspeed

In [3]:
!pip3 install transformers==3.5.0

     |████████████████████████████████| 1.3MB 5.2MB/s 
     |████████████████████████████████| 2.9MB 15.7MB/s 
     |████████████████████████████████| 901kB 34.7MB/s 
     |████████████████████████████████| 1.1MB 52.1MB/s 


In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [ ]:
!sh setup.sh
!git clone  https://github.com/sberbank-ai/ru-gpts

File: /usr/local/lib/python3.7/dist-packages/transformers/trainer_pt_utils.py

> `if version.parse(torch.__version__) <= version.parse("1.4.1") or version.parse(torch.__version__) > version.parse("1.7.0"):`

## Download files

Here we already need the translated data to tune a model

In [ ]:
! gdown https://drive.google.com/uc?id=1dB3X-Wx8qU_5nDG_pxAmLvo5H_sgnHrE
! gdown https://drive.google.com/uc?id=1bJo8TagTGKa0uyppQRqsHrKHyYO5tcZc
! gdown https://drive.google.com/uc?id=11lqipq6ggrgCk8bVxQ4-uuPVMCKN5ebU

import pandas as pd
df_dev_google = pd.read_csv('/content/wiki_dev_cleaned_translated_sd.csv')
df_test_google = pd.read_csv('/content/wiki_test_cleaned_translated_sd.csv')
df_train_google = pd.read_csv('/content/wiki_train_cleaned_translated_sd.csv')

## Prepare data for parallel
We use custom implementation of distributed dataset. For training and evaluating we should specify file `file.list` with list of paths to txt files. All files from `file.list` will be splitted between aviable GPUs. The logic of splitting is described by the following code:

```python
shard_size = len(files) // world_size
shard_start = rank * shard_size
shard_end = (rank + 1) * shard_size
files = files[shard_start:shard_end]
```

For more details please see full code of dataset: `src.dataset_rugpt3.RuGpt3TextDataset`.

In [ ]:
with open('train.txt', 'w+') as f:
  for i,j in list(zip(df_train_google.target_x.values, df_train_google.target_y.values)):
    f.write('<s>'+i+'\n'+j+'\n') # +'<\s>'

with open('valid.txt', 'w+') as f:
  for i,j in list(zip(df_dev_google.target_x.values, df_dev_google.target_y.values)):
    f.write('<s>'+i+'\n'+j+'\n') # +'<\s>'

!echo /content/train.txt > train.list
!echo /content/valid.txt > valid.list

## Train
Load model from Huggingface and finetune on essays.

This will take arount ten minutes.

In [ ]:
! rm -r /content/model
! rm -r /content/model_hf

In [ ]:
!export PYTHONPATH=${PYTHONPATH}:${HOME}/ru-gpts

!python ru-gpts/pretrain_gpt3.py \
  --train-data-path "/content/train.list" \
  --test-data-path "/content/valid.list" \
  --max-files-per-process 100 \
  --logging-dir="log" \
  --save model \
  --load-huggingface sberbank-ai/rugpt3small_based_on_gpt2 \
  --save-interval 1000 \
  --log-interval 100 \
  --eval-interval 1000 \
  --eval-iters 100 \
  --model-parallel-size 1 \
  --num-layers 12 \
  --hidden-size 768 \
  --num-attention-heads 12 \
  --batch-size 1 \
  --seq-length 512 \
  --max-position-embeddings 2048 \
  --train-iters 2000 \
  --resume-dataloader \
  --distributed-backend "nccl" \
  --lr 0.00015 \
  --lr-decay-style "cosine" \
  --lr-decay-iters 3200 \
  --clip-grad 0.5 \
  --warmup .004


2021-04-13 16:44:11.667377: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
using world size: 1 and model-parallel size: 1 
 > using dynamic loss scaling
> initializing model parallel with size 1
Pretrain GPT3 model
arguments:
  attention_dropout ............ 0.1
  num_attention_heads .......... 12
  hidden_size .................. 768
  intermediate_size ............ None
  num_layers ................... 12
  layernorm_epsilon ............ 1e-05
  hidden_dropout ............... 0.1
  max_position_embeddings ...... 2048
  vocab_size ................... 30522
  deep_init .................... False
  make_vocab_size_divisible_by . 8
  cpu_optimizer ................ False
  cpu_torch_adam ............... False
  sparse_mode .................. all
  fp16 ......................... False
  fp32_embedding ............... False
  fp32_layernorm ............... False
  fp32_tokentypes .............. False
  fp32_allreduce ....

At the end of training output should be something like this:

"-----------------------------------------------------------------------------------------

 validation loss at the end of training for test data | LM loss: 2.7927 | LM PPL: 16.325
 
-----------------------------------------------------------------------------------------"

In [ ]:
# ! cp -r /content/model /content/drive/MyDrive/SImplification_models

## Generate

Load pretrained model from dir and generate.

In [ ]:
!export PYTHONPATH=${PYTHONPATH}:${HOME}/ru-gpts

!python ru-gpts/generate_samples.py \
  --load /content/drive/MyDrive/SImplification_models/model \
  --model-parallel-size 1 \
  --num-layers 12 \
  --hidden-size 768 \
  --num-attention-heads 12 \
  --batch-size 1 \
  --seq-length 50 \
  --max-position-embeddings 2048 \
  --distributed-backend "nccl" \
  --tokenizer-path sberbank-ai/rugpt3small_based_on_gpt2 \
  --no-load-optim


### Convert checkpoint to Huggingface format

In [ ]:
# /content/model/

In [ ]:
!export PYTHONPATH=${PYTHONPATH}:${HOME}/ru-gpts

!python ru-gpts/convert2huggingface.py \
  --load model \
  --model-parallel-size 1 \
  --num-layers 12 \
  --hidden-size 768 \
  --num-attention-heads 12 \
  --max-position-embeddings 2048 \
  --tokenizer-path sberbank-ai/rugpt3small_based_on_gpt2 \
  --no-load-optim \
  --export-huggingface /content/drive/MyDrive/SImplification_models/model_hf


In [ ]:
!ls model_hf

#### Test load

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import re

def create_model_and_tok(
        model_name="/content/drive/MyDrive/SImplification_models/model_hf" #/content/drive/MyDrive/SImplification_models/
):
    print('loading from {}'.format(model_name))
    gpt_model = GPT2LMHeadModel.from_pretrained(model_name)
    gpt_tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    gpt_model.cuda()
    gpt_model.eval()
    gpt_tokenizer.padding_side = "left"
    gpt_tokenizer.pad_token = '[PAD]'
    gpt_tokenizer.encoder['[PAD]'] = 50256
    gpt_model.config.pad_token_id = gpt_model.config.eos_token_id
    return gpt_model, gpt_tokenizer


def batch_generator(
        list_of_sentences,
        size=16
):
    num_batch = len(list_of_sentences)//size
    for index in range(num_batch):
        yield list_of_sentences[index*size:(index+1)*size]
    yield list_of_sentences[num_batch*size:]


def get_outputs(
        model,
        tokenizer,
        list_of_sentences,
        seq_len=11, 
        batch_size=16
):
    result = []
    for batch in batch_generator(list_of_sentences, batch_size):
        max_length = len(max(list_of_sentences, key=lambda x: x.split()).split()) + 1
        encodings_dict = tokenizer.batch_encode_plus(batch, max_length=max_length, pad_to_max_length=True, add_special_tokens=False)
        input_ids = torch.tensor(encodings_dict['input_ids']).cuda()
        attn_mask = torch.tensor(encodings_dict['attention_mask']).cuda()

        outputs = model.generate(input_ids,
                                 attention_mask=attn_mask,
                                 do_sample=True,
                                 max_length=40, #1000+ max_length,
                                 top_k=10,
                                 top_p=0.95,
                                 repetition_penalty=5.0,
                                 num_return_sequences=1)
                                 # no_repeat_ngram_size=3)

        outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
        outputs = [text[:re.search(r'<\\?s?>?', text).start()].strip() if  re.search(r'<\\?s?>?', text) is not None else text.strip() for text  in outputs]
        result.extend(outputs)
    return result


if __name__ == '__main__':
    prompt_text = [
        "Британская транспортная комиссия (BTC) была создана послевоенным лейбористским правительством Клемента Эттли в рамках его программы национализации для надзора за железными дорогами, каналами и автомобильными грузовыми перевозками в Великобритании (в Северной Ирландии было отдельное транспортное управление Ольстера).",
        "Отношения Фицджеральда с Гранцем еще больше укрепились, когда он стал ее менеджером, хотя прошло почти десять лет, прежде чем он смог записать ее на одном из своих многочисленных лейблов.",
        "Они вымерли на материке, а оставшиеся популяции были ограничены 32 прибрежными островами до первого выпуска с материка в сильно огороженный и находящийся под наблюдением заповедник Карори в 2005 году.",
        "Вместе с Bryozoa и Brachiopoda форониды принадлежат к лофофоратам, которые иногда рассматриваются как один тип."]

    model, tokenizer = create_model_and_tok()
    texts = get_outputs(model, tokenizer, prompt_text)
    for text in texts:
        print(text)



loading from /content/drive/MyDrive/SImplification_models/model_hf


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Keyword arguments {'add_space_before_punct_symbol': True} not recognized.
Keyword arguments {'add_sp

Британская транспортная комиссия (BTC) была создана послевоенным лейбористским правительством Клемента Эттли в рамках его программы национализации, а затем преобразована из Британской транспортной комиссии
Отношения Фицджеральда с Гранцем еще больше укрепились, когда он стал ее менеджером, хотя прошло почти десять лет, прежде чем она перешла на работу в Microsoft.
Они вымерли на материке, а оставшиеся популяции были ограничены 32 прибрежными островами до первого выпуска с материка в сильно огороженный и густонаселенный регион Северной Америки.
Вместе с Bryozoa и Brachiopoda форониды принадлежат к лофофоратам, которые иногда рассматриваются как этологи.


# Look at the data

In [ ]:
model, tokenizer = create_model_and_tok()
texts = get_outputs(model, tokenizer, test_data['INPUT:source'], batch_size=32)
test_data['gpt_bs'] = texts
test_data.sample(3)

,Unnamed: 0,INPUT:source,OUTPUT:output,trunctuation_bs,gpt_bs
2595,7703,Рамзан Кадыров назвал возрождение многонационального сообщества республики одной из приоритетных задач нового руководства республики.,"Рамзан Кадыров считает, что главная задача руководства - возродить многонациональное общество республики.",Рамзан Кадыров назвал возрождение многонационального сообщества республики одной.,Рамзан Кадыров назвал возрождение многонационального сообщества республики одной из приоритетных задач.
3175,9354,"Улучшение оксигенации лёгких, наблюдаемое при интенсификации дыхания в разрежённом воздухе горных курортов, способствует торможению роста и размножения микобактерий.","Улучшение обогащения лёгких кислородом, которое наблюдается при усилении функций дыхания в разрежённом воздухе горных курортов, способствует также размножению и росту микобактерий.","Улучшение оксигенации лёгких, наблюдаемое при интенсификации дыхания в разрежённом воздухе.","Улучшение оксигенации лёгких, наблюдаемое при интенсификации окиснения урана (AO), является важным механизмом очистки и удаления отходов в легких."
808,2391,"Вторично одичавшие кошки часто живут уединённо и охотятся в одиночку, но иногда образуют небольшие колонии из нескольких самок с котятами.","Как правило, вторично одичавшие кошки и живут и охотятся в одиночку, но бывает, что иногда они образуют отдельные поселения, в которые входят несколько самок и их котята.","Вторично одичавшие кошки часто живут уединённо и охотятся в одиночку, но иногда.",Вторично одичавшие кошки часто живут уединённо и охотятся в дикой природе.


## SAVING BASELINES....

In [3]:
#! gdown https://drive.google.com/uc?id=1IVz3XC8Rm7hQCyx3xCcABjhaENrKABvF
test_data = pd.read_csv('/content/test_baselines.csv', sep='\t')
# EXPERIMENT
#test_data = pd.read_csv('/content/test_data_new.csv', sep='\t')
test_data['INPUT:source'] = test_data['INPUT:source'].apply(lambda x: re.sub(r'[\t\n\r\f\v]+', "",x))
test_data['OUTPUT:output'] = test_data['OUTPUT:output'].apply(lambda x: re.sub(r'[\t\n\r\f\v]+', "",x))
test_data.to_csv('test_data_new.csv', index=False, sep='\t')
test_data = pd.read_csv('/content/test_data_new.csv', sep='\t')


# Environment setup and Necessary functions

In [ ]:
! git clone https://github.com/feralvam/easse
! git clone https://github.com/Andoree/sent_simplification.git
%cp /content/sent_simplification/sari.py /content/easse/easse

%cd easse
! pip install .

In [1]:
%cd /content

/content


In [4]:
! mkdir prepared_data
! mkdir preds

# Evaluation

### BLEU

In [5]:
# function retrieved from easse code
import numpy as np
from typing import List
import sacrebleu
import easse.utils.preprocessing as utils_prep
from collections import OrderedDict


def corpus_bleu(
    sys_sents: List[str],
    refs_sents: List[List[str]],
    smooth_method: str = 'exp',
    smooth_value: float = None,
    force: bool = True,
    lowercase: bool = False,
    tokenizer: str = '13a',
    use_effective_order: bool = False,
):

    sys_sents = [utils_prep.normalize(sent, lowercase, tokenizer) for sent in sys_sents]
    refs_sents = [[utils_prep.normalize(sent, lowercase, tokenizer) for sent in ref_sents] for ref_sents in refs_sents]

    return sacrebleu.corpus_bleu(
        sys_sents,
        refs_sents,
        smooth_method,
        smooth_value,
        force,
        lowercase=False,
        tokenize='none',
        use_effective_order=use_effective_order,
    ).score


def sentence_bleu(
    sys_sent: str,
    ref_sents: List[str],
    smooth_method: str = 'floor',
    smooth_value: float = None,
    lowercase: bool = False,
    tokenizer: str = '13a',
    use_effective_order: bool = True,
):

    return corpus_bleu(
        [sys_sent],
        [[ref] for ref in ref_sents],
        smooth_method,
        smooth_value,
        force=True,
        lowercase=lowercase,
        tokenizer=tokenizer,
        use_effective_order=use_effective_order,
    )


def corpus_averaged_sentence_bleu(
    sys_sents: List[str],
    refs_sents: List[List[str]],
    smooth_method: str = 'floor',
    smooth_value: float = None,
    lowercase: bool = False,
    tokenizer: str = '13a',
    use_effective_order: bool = True,
):

    scores = []
    for sys_sent, *ref_sents in zip(sys_sents, *refs_sents):
        scores.append(
            sentence_bleu(
                sys_sent,
                ref_sents,
                smooth_method,
                smooth_value,
                lowercase=lowercase,
                tokenizer=tokenizer,
                use_effective_order=use_effective_order,
            )
        )
    return np.mean(scores)


### Flesch Kincaid Grade Level

In [6]:
# https://github.com/infoculture/plainrussian
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from math import sqrt
import csv


from numpy import mean, arange


# Russian sounds and characters
RU_CONSONANTS_LOW = [u'к', u'п', u'с', u'т', u'ф', u'х', u'ц', u'ч', u'ш', u'щ']
RU_CONSONANTS_HIGH = [u'б', u'в', u'г', u'д', u'ж', u'з']
RU_CONSONANTS_SONOR = [u'л', u'м', u'н', u'р']
RU_CONSONANTS_YET = [u'й']

RU_CONSONANTS = RU_CONSONANTS_HIGH + RU_CONSONANTS_LOW + RU_CONSONANTS_SONOR + RU_CONSONANTS_YET
RU_VOWELS = [u'а', u'е', u'и', u'у', u'о', u'я', u'ё', u'э', u'ю', u'я', u'ы']
RU_MARKS = [u'ь', u'ъ']
SENTENCE_SPLITTERS = [u'.', u'?', u'!']
RU_LETTERS = RU_CONSONANTS + RU_MARKS + RU_VOWELS
SPACES = [u' ', u'\t']

# List of prepared texts

GRADE_TEXT = {
    1: u'1 - 3-й класс (возраст примерно: 6-8 лет)',
    2: u'1 - 3-й класс (возраст примерно: 6-8 лет)',
    3: u'1 - 3-й класс (возраст примерно: 6-8 лет)',
    4: u'4 - 6-й класс (возраст примерно: 9-11 лет)',
    5: u'4 - 6-й класс (возраст примерно: 9-11 лет)',
    6: u'4 - 6-й класс (возраст примерно: 9-11 лет)',
    7: u'7 - 9-й класс (возраст примерно: 12-14 лет)',
    8: u'7 - 9-й класс (возраст примерно: 12-14 лет)',
    9: u'7 - 9-й класс (возраст примерно: 12-14 лет)',
    10: u'10 - 11-й класс (возраст примерно: 15-16 лет)',
    11: u'10 - 11-й класс (возраст примерно: 15-16 лет)',
    12: u'1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
    13: u'1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
    14: u'1 - 3 курсы ВУЗа (возраст примерно: 17-19 лет)',
    15: u'4 - 6 курсы ВУЗа (возраст примерно: 20-22 лет)',
    16: u'4 - 6 курсы ВУЗа (возраст примерно: 20-22 лет)',
    17: u'4 - 6 курсы ВУЗа (возраст примерно: 20-22 лет)',
}

POST_GRADE_TEXT_18_24 = u'Аспирантура, второе высшее образование, phD'


def calc_SMOG(n_psyl, n_sent):
    """Метрика SMOG для английского языка"""
    n = 1.0430 * sqrt((float(30.0) / n_sent) * n_psyl) + 3.1291
    return n

def calc_Gunning_fog(n_psyl, n_words, n_sent):
    """Метрика Gunning fog для английского языка"""
    n = 0.4 * ((float(n_words)/ n_sent) + 100 * (float(n_psyl) / n_words))
    return n

def calc_Dale_Chale(n_psyl, n_words, n_sent):
    """Метрика Dale Chale для английского языка"""
    n = 0.1579 * (100.0 * n_psyl / n_words) + 0.0496 * (float(n_words) / n_sent)
    return n

def calc_Flesh_Kincaid(n_syllabes, n_words, n_sent):
    """Метрика Flesh Kincaid для английского языка"""
    n = 206.835 - 1.015 * (float(n_words) / n_sent) - 84.6 * (float(n_syllabes) / n_words)
    return n


def calc_Flesh_Kincaid_rus(n_syllabes, n_words, n_sent):
    """Метрика Flesh Kincaid для русского языка"""
    n = 220.755 - 1.315 * (float(n_words) / n_sent) - 50.1 * (float(n_syllabes) / n_words)
    return n

def calc_Flesh_Kincaid_Grade_rus(n_syllabes, n_words, n_sent):
    """Метрика Flesh Kincaid Grade для русского языка"""
#    n = 0.59 * (float(n_words) / n_sent) + 6.2 * (float(n_syllabes) / n_words) - 16.59
    n = 0.49 * (float(n_words) / n_sent) + 7.3 * (float(n_syllabes) / n_words) - 16.59
    return n



def calc_Flesh_Kincaid_Grade_rus_adapted(n_syllabes, n_words, n_sent, X, Y, Z):
    """Метрика Flesh Kincaid Grade для русского языка с параметрами"""
#    n = 0.59 * (float(n_words) / n_sent) + 6.2 * (float(n_syllabes) / n_words) - 16.59
    if n_words == 0 or n_sent == 0: return 0
    n = X * (float(n_words) / n_sent) + Y * (float(n_syllabes) / n_words) - Z
    return n


#X_GRADE = 0.186
#Y_GRADE = 7.21
#Z_GRADE = 15.443

# Flesh Kinkaid Grade константы. Подробнее http://en.wikipedia.org/wiki/Flesch%E2%80%93Kincaid_readability_tests
FLG_X_GRADE = 0.318
FLG_Y_GRADE = 14.2
FLG_Z_GRADE = 30.5

def calc_Flesh_Kincaid_Grade_rus_flex(n_syllabes, n_words, n_sent):
    """Метрика Flesh Kincaid Grade для русского языка с константными параметрами"""
    if n_words == 0 or n_sent == 0: return 0
    n = FLG_X_GRADE * (float(n_words) / n_sent) + FLG_Y_GRADE * (float(n_syllabes) / n_words) - FLG_Z_GRADE
    return n


# Coleman Liau константы. Подробнее http://en.wikipedia.org/wiki/Coleman%E2%80%93Liau_index

CLI_X_GRADE = 0.055
CLI_Y_GRADE = 0.35
CLI_Z_GRADE = 20.33


def calc_Coleman_Liau_index_adapted(n_letters, n_words, n_sent, x, y, z):
    """ Метрика Coleman Liau для русского языка с адаптированными параметрами """
    if n_words == 0: return 0
    n = x * (n_letters * (100.0 / n_words)) - y * (n_sent * (100.0 / n_words)) - z
    return n

def calc_Coleman_Liau_index(n_letters, n_words, n_sent):
    """ Метрика Coleman Liau для русского языка с константными параметрами """
    if n_words == 0: return 0
    n = CLI_X_GRADE * (n_letters * (100.0 / n_words)) - CLI_Y_GRADE * (n_sent * (100.0 / n_words)) - CLI_Z_GRADE
    return n


# Константы SMOG Index http://en.wikipedia.org/wiki/SMOG
SMOG_X_GRADE = 1.1
SMOG_Y_GRADE = 64.6
SMOG_Z_GRADE = 0.05

def calc_SMOG_index(n_psyl, n_sent):
    """Метрика SMOG для русского языка с константными параментрами"""
    n = SMOG_X_GRADE * sqrt((float(SMOG_Y_GRADE) / n_sent) * n_psyl) + SMOG_Z_GRADE
    return n

def calc_SMOG_index_adapted(n_psyl, n_sent, x, y, z):
    """Метрика SMOG для русского языка адаптированная с коэффициентами"""
    n = x * sqrt((float(y) / n_sent) * n_psyl) + z
    return n

DC_X_GRADE = 0.552
DC_Y_GRADE = 0.273

def calc_Dale_Chale_index(n_psyl, n_words, n_sent):
    """Метрика Dale Chale для русского языка с константным параметрами"""
    n = DC_X_GRADE * (100.0 * n_psyl / n_words) + DC_Y_GRADE * (float(n_words) / n_sent)
    return n


def calc_Dale_Chale_adapted(n_psyl, n_words, n_sent, x, y):
    """Метрика Dale Chale для русского языка с адаптированными параметрами"""
    n = x * (100.0 * n_psyl / n_words) + y * (float(n_words) / n_sent)
    return n

ARI_X_GRADE = 6.26
ARI_Y_GRADE = 0.2805
ARI_Z_GRADE = 31.04


def calc_ARI_index_adapted(n_letters, n_words, n_sent, x, y, z):
    """ Метрика Automated Readability Index (ARI) для русского языка с адаптированными параметрами """
    if n_words == 0 or n_sent == 0: return 0
    n = x * (float(n_letters) / n_words) + y * (float(n_words) / n_sent) - z
    return n

def calc_ARI_index(n_letters, n_words, n_sent):
    """ Метрика Automated Readability Index (ARI) для русского языка с константными параметрами """
    if n_words == 0 or n_sent == 0: return 0
    n = ARI_X_GRADE * (float(n_letters) / n_words) + ARI_Y_GRADE * (float(n_words) / n_sent) - ARI_Z_GRADE
    return n


def load_words(filename):
    """Load words from filename"""
    words = []
    f = open(filename, 'r')
    for l in f:
        words.append(l.strip().decode('utf8'))
    f.close()
    return words

#FAM_WORDS = load_words('1norm50000.txt')

bad_chars = '(){}<>"\'!?,.:;'


def calc_text_metrics(filename, verbose=True):
    """Расчет метрик"""
    f = open(filename, 'r')
    text = f.read().decode('utf8')    
    f.close()
    return calc_readability_metrics(text, verbose)


# Number of syllabes for long words
COMPLEX_SYL_FACTOR = 4

def calc_readability_metrics(text, verbose=True):
    sentences = 0
    chars = 0
    spaces = 0
    letters = 0
    syllabes = 0
    words = 0
    complex_words = 0
    simple_words = 0
    wsyllabes = {}

    wordStart = False
    for l in text.splitlines():
        chars += len(l)
#        l = l.decode('utf8')
        for ch in l:
            if ch in SENTENCE_SPLITTERS:
                sentences += 1
            if ch in SPACES:
                spaces += 1

        for w in l.split():
            has_syl = False
            wsyl = 0
#            if len(w) > 1: words += 1
            for ch in w:
                if ch in RU_LETTERS:
                    letters += 1
                if ch in RU_VOWELS:
                    syllabes += 1
                    has_syl = True
                    wsyl += 1
            if wsyl > COMPLEX_SYL_FACTOR:
                complex_words += 1
            elif wsyl < COMPLEX_SYL_FACTOR+1 and wsyl > 0:
                simple_words += 1
            if has_syl:
                words += 1
                v = wsyllabes.get(str(wsyl), 0)
                wsyllabes[str(wsyl)] = v + 1
    metrics = {'c_share': float(complex_words) * 100 / words if words > 0 else 0,
               'avg_slen' : float(words) / sentences if sentences > 0 else 0,
               'avg_syl' : float(syllabes) / words if words > 0 else 0,
               'n_syllabes': syllabes,
               'n_words' : words,
               'n_sentences': sentences,
               'n_complex_words': complex_words,
               'n_simple_words' : simple_words,
               'chars': chars,
               'letters' : letters,
               'spaces' : spaces,
               'index_fk_rus': calc_Flesh_Kincaid_Grade_rus_flex(syllabes, words, sentences),
               'index_cl_rus' : calc_Coleman_Liau_index(letters, words, sentences),
               'index_dc_rus' : calc_Dale_Chale_index(complex_words, words, sentences),
               'index_SMOG_rus' : calc_SMOG_index(complex_words, sentences),
               'index_ari_rus' : calc_ARI_index(letters, words, sentences),
#               'index_fk_rus': calc_Flesh_Kincaid_Grade_rus(syllabes, words, sentences),
               'wsyllabes' : wsyllabes
    }
    del text
    return metrics

### Cosine similarity

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

# shape should be [1, something (768, ex)]

# import numpy as np
# def cs(a, b):
#   return (a @ b.T)/(np.linalg.norm(a)*np.linalg.norm(b))

def calc_cos_sim(df, model,tok, x, y, column_name):
    Cos_sim= []
    for index, row in df.iterrows():
        
        # original
          sentence_A = tok.encode(row[x], padding='max_length', max_length=50, truncation=True, return_tensors='pt')
          sentence_A = sentence_A.to(device)
          output = model(sentence_A)
          sent_emb = output[-1][0]
          emb_source = sent_emb.mean(axis=1)
          emb_source = emb_source.cpu().detach().numpy()

          sentence_B = tok.encode(row[y], padding='max_length', max_length=50, truncation=True, return_tensors='pt')
          sentence_B = sentence_B.to(device)
          output = model(sentence_B)
          sent_emb = output[-1][0]
          emb_target= sent_emb.mean(axis=1)
          emb_target = emb_target.cpu().detach().numpy()

          cos_val = cosine_similarity(emb_source.reshape(emb_source.shape[0], -1), emb_target.reshape(emb_target.shape[0], -1))[0][0]
          Cos_sim.append(cos_val)
    df[column_name] = Cos_sim

### Grammar

In [8]:
def get_mistakes_summary(df_test, x):
    test = list(df_test[x].values)
    matches = []
    for i in test:
      matches.extend(tool.check(i))

    categories = set([i.category for i in matches])

    categories = {i:0 for i in categories}

    for i in matches:
      categories[i.category]+=1

    return categories

In [10]:
! python /content/sent_simplification/refs_to_easse_format.py \
--input_path /content/test_data_new.csv \
--output_dataset_name test_ref_data \
--src_column "INPUT:source" \
--trg_column "OUTPUT:output" \
--output_dir /content/prepared_data

1000
3406
3406
Overall number of references: 3406


## Trunctuation baseline

SARI

In [13]:
sentences = [i for i in test_data.trunctuation_bs.values]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction__tb.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [15]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction__tb.hyp -q

{'sari': 29.097, 'quality_estimation': {'Compression ratio': 0.584, 'Sentence splits': 0.992, 'Levenshtein similarity': 0.736, 'Exact copies': 0.0, 'Additions proportion': 0.0, 'Deletions proportion': 0.395, 'Lexical complexity score': 10.447}}


### BLEU

In [17]:
# make lists of src, dst
src_column, trg_column = "trunctuation_bs", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in test_data.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)

corpus_bleu(src, refs)

30.769443856834286

### Flesh-Kincaid Grade level

In [19]:
calc_readability_metrics(' '.join(list(test_data["trunctuation_bs"].values)))

{'avg_slen': 9.263492063492064,
 'avg_syl': 3.024985980434918,
 'c_share': 17.135023989033584,
 'chars': 276350,
 'index_SMOG_rus': 11.188806752656046,
 'index_ari_rus': 15.801619692667394,
 'index_cl_rus': 14.763557230980126,
 'index_dc_rus': 11.987466575279871,
 'index_fk_rus': 15.40059139836631,
 'letters': 226856,
 'n_complex_words': 5500,
 'n_sentences': 3465,
 'n_simple_words': 26598,
 'n_syllabes': 97096,
 'n_words': 32098,
 'spaces': 34985,
 'wsyllabes': {'1': 5557,
  '10': 17,
  '11': 6,
  '2': 7469,
  '3': 7945,
  '4': 5627,
  '5': 3508,
  '6': 1443,
  '7': 405,
  '8': 100,
  '9': 21}}

### Cosine similarity


```
if version.parse(torch.__version__) <= version.parse("1.4.1") or version.parse(torch.__version__) > version.parse("1.7.0"):
    SAVE_STATE_WARNING = ""
```



In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaModel, AutoConfig, AutoTokenizer, AutoModelForMaskedLM
device = "cuda" if torch.cuda.is_available() else "cpu"
config = AutoConfig.from_pretrained("DeepPavlov/rubert-base-cased") # "roberta-base" 'xlm-mlm-100-1280' 'xlm-roberta-base' 'bert-base-multilingual-cased'
config.output_hidden_states = True

tok = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("DeepPavlov/rubert-base-cased", config=config)
model.to(device)

In [29]:
calc_cos_sim(test_data, model, tok, 'INPUT:source', "trunctuation_bs", 'cos_sim_tb')
test_data.cos_sim_tb.mean()

0.9537958732893099

### Grammar

In [30]:
tool = language_tool_python.LanguageTool('ru')

Unzipping /tmp/tmp3ou51l93.zip to /root/.cache/language_tool_python.
Downloaded https://www.languagetool.org/download/LanguageTool-5.2.zip to /root/.cache/language_tool_python.


In [32]:
errors = get_mistakes_summary(test_data, 'trunctuation_bs')
errors

{'GRAMMAR': 50,
 'LOGIC': 22,
 'PUNCTUATION': 43,
 'STYLE': 203,
 'TYPOGRAPHY': 487,
 'TYPOS': 441}

## GPT baseline

### SARI

In [37]:
sentences = [i for i in test_data.gpt_bs.values]

lt = list()
st = set()
for i in sentences:
  if i not in st:
    lt.append(i)
    st.add(i)

with open('/content/model_prediction__gpt.hyp', 'w') as f:
  for i in lt:
    f.write(i+'\n')

In [38]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/model_prediction__gpt.hyp -q

{'sari': 32.487, 'quality_estimation': {'Compression ratio': 0.958, 'Sentence splits': 1.011, 'Levenshtein similarity': 0.372, 'Exact copies': 0.0, 'Additions proportion': 0.745, 'Deletions proportion': 0.795, 'Lexical complexity score': 10.215}}


### BLEU

In [33]:
# make lists of src, dst
src_column, trg_column = "gpt_bs", "OUTPUT:output"
data_dict = OrderedDict()
for ind, row in test_data.iterrows():
        source_sentence = row[src_column]
        reference_sentence = row[trg_column]
        if data_dict.get(source_sentence) is None:
            data_dict[source_sentence] = []
        data_dict[source_sentence].append(reference_sentence)
max_num_reference_sentences = max([len(x) for x in data_dict.values()])

src, dst = [i for i in data_dict.keys()], [i for i in data_dict.values()]
for i, j in enumerate(src):
   if len(dst[i]) < max_num_reference_sentences:
    dst[i] = dst[i] + [dst[i][0]]*(max_num_reference_sentences - len(dst[i]))

refs = []
for j in range(len(dst[1])):
  ref = []
  for i in dst:
    ref.append(i[j])
  refs.append(ref)

corpus_bleu(src, refs)

10.01418601237403

### FKLG

In [34]:
calc_readability_metrics(' '.join(list(test_data["gpt_bs"].values)))

{'avg_slen': 15.020669577874818,
 'avg_syl': 2.9009419334832156,
 'c_share': 14.880998527017598,
 'chars': 425445,
 'index_SMOG_rus': 13.268118084885156,
 'index_ari_rus': 15.606770178792509,
 'index_cl_rus': 14.621682300953559,
 'index_dc_rus': 12.31495398167354,
 'index_fk_rus': 15.469948381225855,
 'letters': 349744,
 'n_complex_words': 7678,
 'n_sentences': 3435,
 'n_simple_words': 43918,
 'n_syllabes': 149677,
 'n_words': 51596,
 'spaces': 56599,
 'wsyllabes': {'1': 9903,
  '10': 15,
  '11': 6,
  '12': 1,
  '13': 1,
  '2': 12861,
  '3': 12665,
  '4': 8489,
  '5': 4920,
  '6': 2069,
  '7': 512,
  '8': 124,
  '9': 30}}

### Cosine similariry

In [35]:
calc_cos_sim(test_data, model, tok, 'INPUT:source', "gpt_bs", 'cos_sim_gpt')
test_data.cos_sim_gpt.mean()

0.9447574653628288

### Grammar

In [36]:
errors = get_mistakes_summary(test_data, 'gpt_bs')
errors

{'CASING': 2,
 'GRAMMAR': 46,
 'LOGIC': 23,
 'MISC': 3,
 'PUNCTUATION': 54,
 'STYLE': 4,
 'TYPOGRAPHY': 76,
 'TYPOS': 697}